# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the pipeline, we will set up an objective function to minimize the percentage of total transaction value lost to fraud. At the end of this demo, we also show how introducing the right objective during the training before is over 4x better than using a generic machine learning metric like AUC.

In [1]:
import evalml
from evalml.objectives import FraudCost

## Configure "Cost of Fraud" 

To optimize the pipelines toward the specific business needs of this model, you can set your own assumptions for the cost of fraud. These parameters are

* `retry_percentage` - what percentage of customers will retry a transaction if it is declined?
* `interchange_fee` - how much of each successfull transaction do you collect?
* `fraud_payout_percentage` - the percentage of fraud will you be unable to collect
* `amount_col` - the column in the data the represents the transaction amount

Using these parameters, EvalML determines attempt to build a pipeline that will minimize the financial loss due to fraud.

In [2]:
fraud_objective = FraudCost(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount',  
)

## Search for best pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [3]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


In [4]:
# select numeric data before running AutoClassifer
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_pipelines=10)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Cost. Lower score is better.

Searching up to 10 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: xgboost, linear_model, random_forest

Testing XGBoost w/ imputation: 100%|██████████| 10/10 [01:05<00:00,  6.53s/it]                    

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the fraud detection objective we defined

In [6]:
clf.rankings.head(3)

,id,pipeline_name,score,high_variance_cv,parameters
0,9,XGBoostPipeline,0.007614,False,"{'eta': 0.6481718720511973, 'min_child_weight'..."
1,2,XGBoostPipeline,0.007614,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
2,8,RFClassificationPipeline,0.007614,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."


to select the best pipeline we can run

In [7]:
best_pipeline = clf.best_pipeline

### Describe pipeline

You can get more details about any pipeline. Including how it performed on other objective functions.

In [8]:
clf.describe_pipeline(clf.rankings.iloc[0]["id"])

************************
* Pipeline Description *
************************

Pipeline Name: XGBoost w/ imputation
Model type: xgboost
Objective: Fraud Cost (lower is better)
Total training time (including CV): 6.1 seconds

Parameters
• eta: 0.6481718720511973
• min_child_weight: 4.314173858564932
• max_depth: 6
• impute_strategy: most_frequent
• percent_features: 0.871312026764351

Cross Validation
               F1  Precision  Recall   AUC  Log Loss  Fraud Cost # Training # Testing
0           0.264      0.152   0.264 0.841     0.192       0.008  13332.000  6666.000
1           0.264      0.152   0.264 0.845     0.191       0.008  13332.000  6666.000
2           0.264      0.152   0.264 0.834     0.202       0.008  13332.000  6666.000
mean        0.264      0.152   0.264 0.840     0.195       0.008          -         -
std         0.000      0.000   0.000 0.006     0.006       0.000          -         -
coef of var 0.000      0.000   0.000 0.007     0.029       0.002          -        

## Evaluate on hold out

Finally, we retrain the best pipeline on all of the training data and evaluate on the holdout

In [9]:
best_pipeline.fit(X_train, y_train)

Now, we can score the pipeline on the hold out data using both the fraud cost score and the AUC.

In [10]:
best_pipeline.score(X_holdout, y_holdout, other_objectives=["auc", fraud_objective])

(0.007625590107402798,
 {'AUC': 0.8497570761248703, 'Fraud Cost': 0.007625590107402798})

## Why optimize for a problem-specific objective?

To demonstrate the importance of optimzing for the right objective, let's search for another pipeline using AUC, a common machine learning metric. After that, we will score the holdout data using the fraud cost objective to see how the best pipelines compare.

In [11]:
clf_auc = evalml.AutoClassifier(objective='auc',
                            max_pipelines=10)

clf_auc.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for AUC. Greater score is better.

Searching up to 10 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: xgboost, linear_model, random_forest

Testing XGBoost w/ imputation: 100%|██████████| 10/10 [01:29<00:00,  8.96s/it]                    

✔ Optimization finished


like before, we can look at the rankings and pick the best pipeline

In [12]:
clf_auc.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,1,RFClassificationPipeline,0.863238,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."
1,9,XGBoostPipeline,0.852239,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
2,3,RFClassificationPipeline,0.847514,False,"{'n_estimators': 369, 'max_depth': 10, 'impute..."
3,8,RFClassificationPipeline,0.846346,False,"{'n_estimators': 715, 'max_depth': 7, 'impute_..."
4,2,XGBoostPipeline,0.845902,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
5,5,RFClassificationPipeline,0.842745,False,"{'n_estimators': 609, 'max_depth': 7, 'impute_..."
6,4,LogisticRegressionPipeline,0.838806,False,"{'penalty': 'l2', 'C': 6.239401330891865, 'imp..."
7,0,LogisticRegressionPipeline,0.838806,False,"{'penalty': 'l2', 'C': 8.444214828324364, 'imp..."
8,7,LogisticRegressionPipeline,0.838806,False,"{'penalty': 'l2', 'C': 8.123565600467177, 'imp..."
9,6,LogisticRegressionPipeline,0.838806,False,"{'penalty': 'l2', 'C': 0.5765626434012575, 'im..."


In [13]:
best_pipeline_auc = clf_auc.best_pipeline

# train on the full training data
best_pipeline_auc.fit(X_train, y_train)

In [14]:
# get the fraud score on holdout data
best_pipeline_auc.score(X_holdout, y_holdout,  other_objectives=["auc", fraud_objective])

(0.8619958322554153,
 {'AUC': 0.8619958322554153, 'Fraud Cost': 0.03432590219090485})

We can see that search where we optimized for AUC, EvalML finds a pipeline that is slightly better then the pipeline optimized for fraud cost. However, this pipelines losses over 3% of the total transaction amount to fraud compared to under 1% when we optimize for it specifically. 

This example highlights how performance in the real world can diverage greatly from machine learning metrics.